# Predizendo a Potabilidade da Água

# Problema

Elaborar uma rede neural artifical (ANN) para predizer se uma amostra de água é considerada potável, com base em medidas contidas no dataset disponível em https://www.kaggle.com/datasets/adityakadiwal/water-potability

Os parâmetros de entrada para o problema são:
- valor do PH
- dureza, causada por cálcio e sais de magnésio
- total de sólidos dissolvidos
- concentração de cloraminas
- concentração de sulfato
- condutividade
- concentração de carbono orgânico
- concentração de trialometanos
- turbidez

E a saída é binária de acordo com que a água seja potável ou não.

# Pacotes

In [1]:
import numpy as np # para computação científica
import tensorflow as tf # para computação númerica nos dados
from tensorflow import keras # para deep learning
import pandas as pd # para trabalhar com análise de dados

2022-12-29 20:01:47.004391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-29 20:01:47.103983: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Pré-processamento dos dados de treino e teste

## Lendo o dataset

In [2]:
data = pd.read_csv('WaterPotability.csv') # lẽ o dataset

data.head(10) # mostra as 10 primeiras linhas do dataset

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
8,NaN,118.988579,14285.583854,7.804174,268.646941,389.375566,12.706049,53.928846,3.595017,0
9,11.180284,227.231469,25484.508491,9.077200,404.041635,563.885481,17.927806,71.976601,4.370562,0


## Removendo linhas que possuem ausência de dados no dataset

A forma de lidar com a ausência de dados foi remover a linha completa onde um ou mais valores nulos (NaN) forem encontrados. A consequência disso é uma enorme redução do dataset disponível.

In [3]:
data.dropna(axis="index", how='any', inplace=True) # remove linhas onde for encontrado valor nulo

data.head(10) # mostra as 10 primeiras linhas do dataset após a remocão

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
9,11.180284,227.231469,25484.508491,9.077200,404.041635,563.885481,17.927806,71.976601,4.370562,0
10,7.360640,165.520797,32452.614409,7.550701,326.624353,425.383419,15.586810,78.740016,3.662292,0
12,7.119824,156.704993,18730.813653,3.606036,282.344050,347.715027,15.929536,79.500778,3.445756,0
15,6.347272,186.732881,41065.234765,9.629596,364.487687,516.743282,11.539781,75.071617,4.376348,0
17,9.181560,273.813807,24041.326280,6.904990,398.350517,477.974642,13.387341,71.457362,4.503661,0


## Separando as entradas e saídas

### Entradas

In [4]:
X = data.drop("Potability", axis=1)

X.head(10)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425
9,11.180284,227.231469,25484.508491,9.077200,404.041635,563.885481,17.927806,71.976601,4.370562
10,7.360640,165.520797,32452.614409,7.550701,326.624353,425.383419,15.586810,78.740016,3.662292
12,7.119824,156.704993,18730.813653,3.606036,282.344050,347.715027,15.929536,79.500778,3.445756
15,6.347272,186.732881,41065.234765,9.629596,364.487687,516.743282,11.539781,75.071617,4.376348
17,9.181560,273.813807,24041.326280,6.904990,398.350517,477.974642,13.387341,71.457362,4.503661


### Saídas

In [5]:
Y = data["Potability"]

Y.head(10)

3     0
4     0
5     0
6     0
7     0
9     0
10    0
12    0
15    0
17    0
Name: Potability, dtype: int64

## Criando conjuntos de treino e teste para as entradas e saídas

In [6]:
from sklearn.model_selection import train_test_split
train_set_X, test_set_X, train_set_Y, test_set_Y = train_test_split(X, Y, test_size=0.20)

print(train_set_X, test_set_X, train_set_Y, test_set_Y, sep="\n")

            ph    Hardness        Solids  Chloramines     Sulfate  \
1170  6.770927  181.818066  26880.220040     5.215862  284.745510   
3129  6.397905  249.969243  22057.051786     4.892717  375.494474   
1499  6.999765  219.143373  16078.459386     7.652135  355.688892   
1784  9.252857  168.040751  50279.262429     5.905056  415.450810   
797   7.368769  213.197019  18452.022995     8.292445  294.344858   
...        ...         ...           ...          ...         ...   
2749  5.913133  184.393156  40847.760146     7.041828  331.959964   
414   5.596628  177.213792  17925.352120     8.435470  303.734166   
129   6.506488  202.285938  17087.076902     6.201983  326.662368   
1036  5.524297  157.142424  26220.174214     6.344585  379.788376   
1869  6.548018  176.006536  38368.012618     7.415054  278.093309   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  
1170    475.548960       14.665441        60.515772   4.910534  
3129    334.269880       14.150363       

## Padronizando os atributos

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_set_X = scaler.fit_transform(train_set_X)
test_set_X  = scaler.fit_transform(test_set_X)

print(train_set_X, test_set_X, sep="\n")

[[-0.20160982 -0.44489924  0.56782933 ...  0.0929792  -0.36591221
   1.1885001 ]
 [-0.43796294  1.64544033  0.0109035  ... -0.06238438  1.91791983
  -0.01690925]
 [-0.05661434  0.69994621 -0.67943786 ...  0.88690633 -0.0813738
  -0.77461896]
 ...
 [-0.36916303  0.18289334 -0.56297393 ...  0.65695205 -1.26094091
  -0.53575695]
 [-0.99149642 -1.20175291  0.49161458 ...  0.76795055 -0.9544134
   0.07786293]
 [-0.34284898 -0.62315104  1.89431186 ...  1.52786596  0.93833791
   2.47963542]]
[[-1.12245524  0.00986732 -0.59350467 ... -1.24416612 -0.42464483
   0.63977294]
 [-0.74747277 -0.58821667 -1.13277741 ... -0.54448127 -0.08201102
   0.08467564]
 [ 0.9582698   1.53728323 -0.87448797 ...  1.15178176 -0.55917717
  -1.37997936]
 ...
 [ 0.13705342  0.42505097  1.06842008 ... -1.12927287 -0.15324727
   0.97900702]
 [ 0.57862389 -1.33148099  0.94556134 ...  1.4953231   0.70830193
  -2.05530535]
 [ 1.58829217  0.18375595 -0.19785743 ...  0.67767462  0.29071066
  -1.26472597]]


## Obtendo número de atributos e exemplos de treinamento

In [8]:
n = train_set_X.shape[1] # número de atributos
m = train_set_X.shape[0] # número de exemplos para treinamento

print ("Número de atributos: n = " + str(n))
print ("Número de exemplos para treinamento: m = " + str(m))

Número de atributos: n = 9
Número de exemplos para treinamento: m = 1608


# Definição do modelo
- Camada de entrada: de acordo com as 9 entradas do problema
- Camada 1: 7 neurônios, função de ativação *ReLu*
- Camada 2 (saída): 1 neurônio, função de ativação *Sigmoid*

In [9]:
inputs = keras.Input(shape=n)
x = keras.layers.Dense(units=3, activation="relu")(inputs)
x = keras.layers.Dense(units=4, activation="relu")(inputs)
outputs = keras.layers.Dense(units=1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 40        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 45
Trainable params: 45
Non-trainable params: 0
_________________________________________________________________


2022-12-29 20:01:49.006217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 20:01:49.041224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 20:01:49.041491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 20:01:49.041971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFl

# Compilação do modelo

In [10]:
model.compile(optimizer="RMSprop", loss="mean_absolute_error", metrics=["accuracy", "Precision", "Recall"])

# Treinamento do modelo

In [11]:
model.fit(train_set_X, train_set_Y, batch_size=32, epochs=500)

Epoch 1/500
51/51 [==============================] - 1s 2ms/step - loss: 0.4606 - accuracy: 0.5964 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/500
51/51 [==============================] - 0s 2ms/step - loss: 0.4554 - accuracy: 0.5964 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/500
51/51 [==============================] - 0s 2ms/step - loss: 0.4507 - accuracy: 0.5964 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/500
51/51 [==============================] - 0s 2ms/step - loss: 0.4460 - accuracy: 0.5964 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/500
51/51 [==============================] - 0s 2ms/step - loss: 0.4419 - accuracy: 0.5964 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/500
51/51 [==============================] - 0s 2ms/step - loss: 0.4381 - accuracy: 0.5964 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/500
51/51 [==============================] - 0s 2ms/step - loss: 0.4348 - accuracy: 0.5964 - precision: 0.0000e+00 - recall: 0.0

# Avaliação do modelo

In [12]:
loss, acc, prec, rec = model.evaluate(test_set_X, test_set_Y)

print("Loss: %.2f" % loss, "\nAccuracy: %.2f" % acc, "\nPrecision: %.2f" % prec, "\nRecall: %.2f" % rec)

13/13 [==============================] - 0s 1ms/step - loss: 0.3562 - accuracy: 0.6452 - precision: 0.6667 - recall: 0.2346
Loss: 0.36 
Accuracy: 0.65 
Precision: 0.67 
Recall: 0.23


# Predição

In [13]:
predictions = model.predict(test_set_X)

print("Predição: ", [round(x[0]) for x in predictions])
print()
print("Correto: ", [round(x) for x in test_set_Y])

13/13 [==============================] - 0s 891us/step
Predição:  [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 